# QA question generator using Deepseek


**Authors**
1. Alfan Dinda Rahmawan (alfan.d.rahmawan@gdplabs.id)

## Install dependencies

In [1]:
%pip install -q langchain=="0.3.0"
%pip install -q langchain-aws=="0.2.7"
%pip install -q langchain_openai=="0.2.14"
%pip install -q boto3=="1.35.71"
%pip install -q pandas=="2.2.2"
%pip install -q tqdm=="4.66.4"
%pip install google-api-python-client==2.100.0 gspread==5.10.0
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Data Preparation

In [2]:
import os
import pandas as pd
import json

from typing import List
from dotenv import load_dotenv

load_dotenv()

GOOGLE_SPREADSHEET_ID: str = "1dDMqrol_DrEMjvLy88IRu2WdHN7T5BU0LrD8ORLuNPI" # put your spreadsheet id here
GOOGLE_SPREADSHEET_URL: str = f"https://docs.google.com/spreadsheets/d/{GOOGLE_SPREADSHEET_ID}/edit?usp=sharing" # put your spreadsheet link here
DATA_TEST_SHEET_NAME: str = "test_data_time_management"

GOOGLE_SHEETS_CLIENT_EMAIL: str = os.getenv('GOOGLE_SHEETS_CLIENT_EMAIL')
GOOGLE_SHEETS_PRIVATE_KEY: str = os.getenv('GOOGLE_SHEETS_PRIVATE_KEY')

### Google Auth

In [2]:
# Google Authentication
from modules.google_sheets_writer import GoogleUtil

PRIVATE_KEY = GOOGLE_SHEETS_PRIVATE_KEY
google: GoogleUtil = GoogleUtil(PRIVATE_KEY, GOOGLE_SHEETS_CLIENT_EMAIL)


In [3]:
## Load Data Test
rows: List[list] = google.retrieve_worksheet(GOOGLE_SPREADSHEET_ID, DATA_TEST_SHEET_NAME)
df_data_test: pd.DataFrame = pd.DataFrame(rows[1:], columns=rows[0])
df_data_test = df_data_test[50:]
display(df_data_test.head(2))

,No,Prompt,Category,Expected SQL Query,Expected Query Result,Expected SQL Query (base from catapa),Expected Query Result (base from catapa),Status,llm_prompt,Local Query Result,Database
50,51,Berapa jumlah karyawan baru yang direkrut seti...,,WITH monthly_employees AS (\n SELECT \n DA...,"[{'bulan': '2023-10', 'jumlah_karyawan_baru': 2}]",,,,,,
51,52,Berapa jumlah karyawan yang memiliki bawahan l...,,SELECT COUNT(DISTINCT manager_id) AS total_man...,[{'total_managers': 4}],,,,,,


## Database Information

In [1]:
import pandas as pd
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from tqdm import tqdm

from modules.database_info.schema import employee_schema, time_management_schema
from modules.database_info.master_data import employee_master_data, time_management_master_data
from modules.database_info.relation import employee_relations, time_management_relations
from modules.database_info.trustee_tables import data_trustee_employee, data_trustee_time_management
from modules.database_info.anonymize_entities import anonymized_entities_description

## Prompt augmentation sql question and sql query

In [4]:
SYSTEM_MESSAGE = """<instructions>
You are an HR SQL augmentation specialist with expertise in creating diverse but RELATED business question-query pairs. Your task is to generate HR analytics questions that explore variations and extensions of the original question while using different analytical techniques.

For each synthetic question-query pair, you will provide:
1. BUSINESS_QUESTION: A question in Indonesian that is related to the core theme of the original question
2. SQL_REASONING: Step-by-step explanation connecting the business question to the SQL solution in Indonesian
3. SQL_QUERY: Clean, executable SQL code that answers the business question

The existing question serves as the central theme. Your generated questions should explore different angles, dimensions, or deeper insights related to this theme.
</instructions>"""

USER_MESSAGE = """I need you to generate {total_augmentations} RELATED but DIVERSE HR business questions and SQL queries based on the existing question.

## DATABASE SCHEMA
{schema}

## DATABASE RELATIONS
{relations}

## MASTER DATA
{master_data}

## DATA TRUSTEE (Sensitive Fields)
{data_trustee_tables}

## ANONYMIZED ENTITIES
{anonymized_entities_description}

## EXISTING QUESTION-QUERY PAIR (THIS DEFINES THE THEME)
### Question
{existing_question}
### Query
{existing_query_pairs}

## TASK
Generate EXACTLY {total_augmentations} questions that are RELATED to the existing question's theme but explore different aspects, angles, or deeper insights. Each example must:

1. MAINTAIN THEMATIC CONNECTION: Each question should relate to the core theme of the existing question (e.g., if the original is about age distribution, generated questions should explore different aspects of age demographics)
2. USE DIFFERENT ANALYTICAL APPROACHES: Each generated question should analyze the theme using a different approach
3. VARY IN COMPLEXITY: Include both simpler and more complex variations
4. EXTEND THE ANALYSIS: Go beyond the basic analysis in the original question to provide deeper or different insights on the same theme

## RELATED VARIATIONS APPROACH

For each question, create variations by:
1. ADDING DIMENSIONS: Segment the core metric by additional variables (department, gender, location, etc.)
2. TEMPORAL ANALYSIS: Explore how the core metric changes over time
3. CORRELATION EXPLORATION: Analyze relationships between the core metric and other variables
4. OUTLIER IDENTIFICATION: Find notable extremes or anomalies in the core metric
5. COMPARATIVE STUDY: Compare the core metric across different groups or categories
6. TREND ANALYSIS: Identify patterns or trajectories in the core metric

## ANALYTICAL APPROACHES TO INCLUDE
Each question should use a DIFFERENT analytical approach related to the theme:
1. SEGMENTATION ANALYSIS: Dividing the core metric by different dimensions
2. TIME SERIES ANALYSIS: Analyzing trends in the core metric over time
3. COMPARATIVE ANALYSIS: Comparing the core metric across different groups
4. CORRELATION ANALYSIS: Examining relationships between the core metric and other variables
5. ANOMALY DETECTION: Identifying outliers or unusual patterns in the core metric
6. DISTRIBUTION ANALYSIS: Examining how the core metric is distributed across different ranges
7. RATIO ANALYSIS: Calculating and comparing ratios related to the core metric
8. PREDICTIVE MODELING: Using the core metric to predict future outcomes
9. COHORT ANALYSIS: Comparing groups based on shared characteristics related to the core metric
10. VARIANCE ANALYSIS: Analyzing the spread or variability of the core metric

## SQL TECHNIQUES TO UTILIZE
Each question should use DIFFERENT SQL techniques, appropriate for the analysis:
- Window functions (ROW_NUMBER, RANK, DENSE_RANK, LAG, LEAD, etc.)
- Common Table Expressions (CTEs) for complex multi-step analysis
- CASE statements for conditional logic and categorization
- Date and time functions for temporal analysis
- Subqueries and correlated subqueries
- Self-joins for comparing records
- Aggregation functions (COUNT, SUM, AVG, MIN, MAX, etc.)
- Statistical functions (PERCENTILE, STDDEV, etc.)
- Pivoting data for different perspectives
- HAVING clauses for filtered aggregations

## FORMAT REQUIREMENTS
- Always include these filtering conditions with placeholders:
  * `organization_id IN ('[ORGANIZATION_IDS]')`
  * `job_level_id IN ('[JOB_LEVEL_IDS]')`
  * `location_id IN ('[LOCATION_IDS]')`
- Use exact format for date functions: `STR_TO_DATE('2023-01-01', '%Y-%m-%d')`
- Include appropriate placeholders in every query

## OUTPUT FORMAT
Provide the output in this JSON structure:

```json
{{
  "augmented_questions": [
    {{
      "question_id": 1,
      "business_question": "<augmented_business_question>",
      "sql_reasoning": "<augmented_sql_reasoning>",
      "required_tables": ["<augmented_required_tables>"],
      "sql_query": "<augmented_sql_query>"
    }},
    // Additional examples with RELATED but DIFFERENT analytical approaches
  ]
}}
```

## VERIFICATION STEPS
Before finalizing your response, verify that:
1. You have generated EXACTLY {total_augmentations} questions
2. EACH question is clearly RELATED to the theme of the existing question
3. EACH question explores a DIFFERENT aspect or dimension of the same theme
4. EACH question uses a DIFFERENT analytical approach
5. EACH question employs DIFFERENT SQL techniques

The goal is to create variations that explore the same theme from different angles, providing a comprehensive set of analytics on the core subject.
"""

### Creates the data generation pipeline

In [5]:
class QnAPair(BaseModel):
    paraphrased_input: str = Field(description="paraphrased_input")
    test_to_sql_query: str = Field(description="test_to_sql_query")

class QnAPairs(BaseModel):
    qna_pairs: List[QnAPair] = Field(description="list of qna pairs")

## OpenAI / DeepSeek
DEEPSEEK_MODEL_NAME = os.getenv("DEEPSEEK_MODEL")
DEEPSEEK_ENDPOINT = os.getenv("DEEPSEEK_ENDPOINT")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

llm = ChatOpenAI(
    model_name=DEEPSEEK_MODEL_NAME,
    temperature=0.7,  # Higher temperature (0.7-0.9) for more creative variations
    openai_api_base=DEEPSEEK_ENDPOINT,
    openai_api_key=DEEPSEEK_API_KEY,
    top_p=0.95,  # Keep high top_p for diverse outputs while filtering unlikely tokens
    seed=42  # Optional: set seed for reproducibility
)

# Create prompt template
system_message_prompt = SystemMessagePromptTemplate.from_template(SYSTEM_MESSAGE)
human_message_prompt = HumanMessagePromptTemplate.from_template(USER_MESSAGE)
prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])
parser = JsonOutputParser(pydantic_object=QnAPairs)
chain = prompt | llm | parser

### Sanity Check

In [7]:
from datetime import datetime

current_date = datetime.now().strftime("%d %B %Y")

database_type = "employee"

business_question = "Berapa total nilai tanpa angka dibelakang koma untuk custom data LINTING_MEREK_SATU"

current_sql_query = """SELECT
    SUM(CAST(employee_variables.value AS DECIMAL(10,0))) AS 'total_linting_merek_satu'
FROM
    employee_variables
    JOIN employee_variable_metadata
        ON employee_variables.employee_variable_metadata_id = employee_variable_metadata.id
    JOIN employees
        ON employee_variables.employee_id = employees.id
    JOIN employment_statuses
        ON employees.id = employment_statuses.employee_id
WHERE
    employee_variable_metadata.name = 'LINTING_MEREK_SATU'
    AND employment_statuses.organization_id IN ([ORGANIZATION_IDS])
    AND employment_statuses.job_level_id IN ([JOB_LEVEL_IDS])
    AND employment_statuses.location_id IN ([LOCATION_IDS]);"""


total_augmentations = 3

if database_type == "employee":
    schema = employee_schema
    relations = employee_relations
    master_data = employee_master_data
    data_trustee_tables = data_trustee_employee
    master_data = employee_master_data
else:
    schema = time_management_schema
    relations = time_management_relations
    master_data = time_management_master_data
    data_trustee_tables = data_trustee_time_management
    master_data = time_management_master_data

response = chain.invoke({
    "schema": schema,
    "relations": relations,
    "master_data": master_data,
    "data_trustee_tables": data_trustee_tables,
    "anonymized_entities_description": anonymized_entities_description,
    "existing_question": business_question,
    "existing_query_pairs": current_sql_query,
    "total_augmentations": total_augmentations
})

if response["augmented_questions"]:
    for qna_pair in response["augmented_questions"]:
        print(qna_pair["business_question"])
        print(qna_pair["sql_query"])
        print("-"*100)

2025-04-11 02:13:55,825 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


Bagaimana distribusi nilai custom data LINTING_MEREK_SATU berdasarkan jenis kelamin karyawan?
SELECT
    ed.gender,
    MIN(CAST(ev.value AS DECIMAL(10,0))) AS min_value,
    MAX(CAST(ev.value AS DECIMAL(10,0))) AS max_value,
    AVG(CAST(ev.value AS DECIMAL(10,0))) AS avg_value,
    COUNT(*) AS count_employees
FROM
    employee_variables ev
    JOIN employee_variable_metadata evm ON ev.employee_variable_metadata_id = evm.id
    JOIN employees e ON ev.employee_id = e.id
    JOIN employment_statuses es ON e.id = es.employee_id
    JOIN employee_details ed ON e.id = ed.employee_id
WHERE
    evm.name = 'LINTING_MEREK_SATU'
    AND es.organization_id IN ('[ORGANIZATION_IDS]')
    AND es.job_level_id IN ('[JOB_LEVEL_IDS]')
    AND es.location_id IN ('[LOCATION_IDS]')
GROUP BY
    ed.gender
ORDER BY
    avg_value DESC;
----------------------------------------------------------------------------------------------------
Bagaimana tren nilai rata-rata custom data LINTING_MEREK_SATU per bulan da

## Generate augmentation synthetics data

In [8]:
from datetime import datetime
import json
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from modules.constants import ColumnName

def generate_augmentations_in_batches(
    df_data_test: pd.DataFrame,
    batch_size: int = 10,
    total_augmentations: int = 3,
    database_type: str = "employee"
):
    """Generate augmentations for test data in batches and save results.

    Args:
        df_data_test (pd.DataFrame): DataFrame containing test data with questions and SQL queries
        batch_size (int): Number of questions to process in each batch
        total_augmentations (int): Number of augmentations to generate per question
        database_type (str): Type of case to process ("employee" or "time_management")

    Returns:
        pd.DataFrame: DataFrame containing all augmented data
    """
    # Create directory for results
    augmented_data_dir = Path("augmented_sql_data_2")
    augmented_data_dir.mkdir(parents=True, exist_ok=True)

    # Define progress file path
    progress_file = augmented_data_dir / "augmentation_progress.json"

    # Prepare master data based on case type

    # Select appropriate schema and data based on case type
    if database_type == "employee":
        schema = employee_schema
        relations = employee_relations
        master_data = employee_master_data
        data_trustee_tables = data_trustee_employee
    else:
        schema = time_management_schema
        relations = time_management_relations
        master_data = time_management_master_data
        data_trustee_tables = data_trustee_time_management

    # Initialize results DataFrame
    all_augmentations = pd.DataFrame(
        columns=[
            ColumnName.NO,
            ColumnName.BASE_PROMPT,
            ColumnName.PROMPT,
            ColumnName.SQL_REASONING,
            ColumnName.REQUIRED_TABLES,
            ColumnName.EXPECTED_SQL_QUERY
        ]
    )

    # Try to load existing progress if available
    processed_indices = set()
    if progress_file.exists():
        try:
            with open(progress_file, "r") as f:
                progress_data = json.load(f)
                all_augmentations = pd.DataFrame(progress_data.get("all_augmentations", []))
                processed_indices = set(progress_data.get("processed_indices", []))
                print(f"Loaded existing progress: {len(processed_indices)}/{len(df_data_test)} questions processed")
        except Exception as e:
            print(f"Error loading progress file: {e}")

    # Process data in batches
    total_rows = len(df_data_test)
    for batch_start in range(0, total_rows, batch_size):
        batch_end = min(batch_start + batch_size, total_rows)
        print(f"Processing batch {batch_start//batch_size + 1}: rows {batch_start} to {batch_end-1}")

        batch_augmentations = []

        # Process each row in the current batch
        for idx in tqdm(range(batch_start, batch_end), desc=f"Batch {batch_start//batch_size + 1}"):
            # Skip already processed indices
            if idx in processed_indices:
                continue

            row = df_data_test.iloc[idx]
            no = row[ColumnName.NO]
            base_business_question = row[ColumnName.PROMPT]
            base_sql_query = row[ColumnName.EXPECTED_SQL_QUERY]

            # Create existing_question_query_pairs for context
            # existing_question_query_pairs = {
            #     "business_question": base_business_question,
            #     "sql_query": base_sql_query
            # }
            try:
                # Generate augmentations
                # response = chain.invoke({
                #     "schema": schema,
                #     "relations": relations,
                #     "master_data": master_data,
                #     "data_trustee_tables": data_trustee_tables,
                #     "anonymized_entities_description": anonymized_entities_description,
                #     "existing_question_query_pairs": existing_question_query_pairs,
                #     "total_augmentations": total_augmentations
                # })

                response = chain.invoke({
                    "schema": schema,
                    "relations": relations,
                    "master_data": master_data,
                    "data_trustee_tables": data_trustee_tables,
                    "anonymized_entities_description": anonymized_entities_description,
                    "existing_question": base_business_question,
                    "existing_query_pairs": base_sql_query,
                    "total_augmentations": total_augmentations
                })

                # Process augmentations
                if response["augmented_questions"]:
                    for aug_idx, qna_pair in enumerate(response["augmented_questions"]):
                        augmentation_no = f"{no}_{aug_idx + 1}"
                        business_question = qna_pair["business_question"]
                        sql_query = qna_pair["sql_query"]
                        sql_reasoning = qna_pair["sql_reasoning"]
                        required_tables = qna_pair["required_tables"]

                        # Add to batch results
                        batch_augmentations.append({
                            ColumnName.NO: augmentation_no,
                            ColumnName.BASE_PROMPT: base_business_question,
                            ColumnName.PROMPT: business_question,
                            ColumnName.SQL_REASONING: sql_reasoning,
                            ColumnName.REQUIRED_TABLES: required_tables,
                            ColumnName.EXPECTED_SQL_QUERY: sql_query
                        })

                # Mark as processed
                processed_indices.add(idx)

            except Exception as e:
                print(f"Error processing row {idx} ({ColumnName.NO}: {no}): {e}")
                # Log the error but continue with next item
                with open(augmented_data_dir / "augmentation_errors.log", "a") as f:
                    f.write(f"Row {idx} ({ColumnName.NO}: {no}) error: {str(e)}\n")

        # Add batch results to all_augmentations
        if batch_augmentations:
            batch_df = pd.DataFrame(batch_augmentations)
            all_augmentations = pd.concat([all_augmentations, batch_df], ignore_index=True)

            # Save progress
            with open(progress_file, "w") as f:
                json.dump({
                    "all_augmentations": all_augmentations.to_dict("records"),
                    "processed_indices": list(processed_indices),
                    "last_updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }, f, indent=2)

            # Save current results to CSV
            all_augmentations.to_csv(augmented_data_dir / f"augmented_data_{database_type}_{len(df_data_test)}_questions_in_progress.csv", index=False)

    # Save final results
    final_csv_path = augmented_data_dir / f"augmented_data_{database_type}_{len(df_data_test)}_questions_complete.csv"
    all_augmentations.to_csv(final_csv_path, index=False)
    print(f"Successfully generated {len(all_augmentations)} augmentations for {len(processed_indices)} questions")
    print(f"Results saved to {final_csv_path}")

    return all_augmentations

# Run the augmentation process
batch_size = 10  # Process 10 questions at a time
total_augmentations = 3  # Generate 3 augmentations per question
# database_type = "employee"  # Use "employee" or "time_management"
database_type = "time_management"  # Use "employee" or "time_management"

augmented_data = generate_augmentations_in_batches(
    df_data_test,
    batch_size=batch_size,
    total_augmentations=total_augmentations,
    database_type=database_type
)

# Display sample of results
print("\nSample of augmented data:")
display(augmented_data.head(7))

Processing batch 1: rows 0 to 9


Batch 1:  20%|██        | 2/10 [01:42<06:46, 50.82s/it]2025-04-11 02:17:49,227 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-11 02:18:00,942 - INFO - Retrying request to /chat/completions in 0.435330 seconds
2025-04-11 02:18:02,048 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
Batch 1: 100%|██████████| 10/10 [09:36<00:00, 57.67s/it]


Processing batch 2: rows 10 to 19


Batch 2: 100%|██████████| 10/10 [09:48<00:00, 58.87s/it]


Processing batch 3: rows 20 to 29


...

Batch 19: 100%|██████████| 10/10 [09:01<00:00, 54.19s/it]


Processing batch 20: rows 190 to 195


Batch 20: 100%|██████████| 6/6 [04:51<00:00, 48.52s/it]


Successfully generated 588 augmentations for 196 questions
Results saved to augmented_sql_data_2/augmented_data_time_management_196_questions_complete.csv

Sample of augmented data:


,No,Base Prompt,Prompt,SQL Reasoning,Required Tables,Expected SQL Query
0,51_1,Berapa jumlah karyawan baru yang direkrut seti...,Bagaimana distribusi karyawan baru pada tahun ...,Pertanyaan ini memerlukan analisis segmentasi ...,"[employees, employee_details, employment_statu...",WITH monthly_employees AS (\n SELECT \n DA...
1,51_2,Berapa jumlah karyawan baru yang direkrut seti...,Bagaimana tingkat retensi karyawan baru yang d...,Pertanyaan ini memerlukan analisis kohort untu...,"[employees, employment_statuses]","WITH q1_recruits AS (\n SELECT \n e.id,\n ..."
2,51_3,Berapa jumlah karyawan baru yang direkrut seti...,Bagaimana perbandingan tingkat rekrutmen karya...,Pertanyaan ini memerlukan analisis komparatif ...,"[employees, employment_statuses, locations]","SELECT \n l.name AS lokasi_kerja,\n COUNT(e...."
3,52_1,Berapa jumlah karyawan yang memiliki bawahan l...,Bagaimana distribusi jumlah bawahan langsung p...,Pertanyaan ini memperluas analisis dengan meli...,"[employees, employment_statuses, organizations]",WITH manager_subordinates AS (\n SELECT \n ...
4,52_2,Berapa jumlah karyawan yang memiliki bawahan l...,Bagaimana tren perubahan jumlah manager dalam ...,Pertanyaan ini menganalisis tren temporal deng...,"[employees, employment_status_histories, emplo...",WITH monthly_managers AS (\n SELECT \n ...
5,52_3,Berapa jumlah karyawan yang memiliki bawahan l...,Berapa rasio manager terhadap total karyawan d...,Pertanyaan ini melakukan analisis komparatif d...,"[employees, employment_statuses, job_levels]",WITH employee_counts AS (\n SELECT \n ...
6,53_1,Berapa persentase karyawan yang terlambat lebi...,Bagaimana tren keterlambatan karyawan (lebih d...,Pertanyaan ini memerlukan analisis time series...,"[presence_entries, attendance_statuses, employ...",WITH monthly_late_data AS (\n SELECT \n or...


In [9]:
augmented_data

,No,Base Prompt,Prompt,SQL Reasoning,Required Tables,Expected SQL Query
0,51_1,Berapa jumlah karyawan baru yang direkrut seti...,Bagaimana distribusi karyawan baru pada tahun ...,Pertanyaan ini memerlukan analisis segmentasi ...,"[employees, employee_details, employment_statu...",WITH monthly_employees AS (\n SELECT \n DA...
1,51_2,Berapa jumlah karyawan baru yang direkrut seti...,Bagaimana tingkat retensi karyawan baru yang d...,Pertanyaan ini memerlukan analisis kohort untu...,"[employees, employment_statuses]","WITH q1_recruits AS (\n SELECT \n e.id,\n ..."
2,51_3,Berapa jumlah karyawan baru yang direkrut seti...,Bagaimana perbandingan tingkat rekrutmen karya...,Pertanyaan ini memerlukan analisis komparatif ...,"[employees, employment_statuses, locations]","SELECT \n l.name AS lokasi_kerja,\n COUNT(e...."
3,52_1,Berapa jumlah karyawan yang memiliki bawahan l...,Bagaimana distribusi jumlah bawahan langsung p...,Pertanyaan ini memperluas analisis dengan meli...,"[employees, employment_statuses, organizations]",WITH manager_subordinates AS (\n SELECT \n ...
4,52_2,Berapa jumlah karyawan yang memiliki bawahan l...,Bagaimana tren perubahan jumlah manager dalam ...,Pertanyaan ini menganalisis tren temporal deng...,"[employees, employment_status_histories, emplo...",WITH monthly_managers AS (\n SELECT \n ...
...,...,...,...,...,...,...
583,245_2,Berapa jumlah karyawan yang memiliki lebih dar...,Apakah ada korelasi antara jumlah bawahan lang...,Pertanyaan ini memerlukan analisis korelasi un...,"[employees, employment_statuses, job_levels]","SELECT jl.name AS job_level, AVG(sub.report_co..."
584,245_3,Berapa jumlah karyawan yang memiliki lebih dar...,Bagaimana tren perubahan jumlah bawahan langsu...,Pertanyaan ini memerlukan analisis time series...,"[employees, employment_statuses, employment_st...",WITH monthly_data AS (SELECT DATE_FORMAT(h.eff...
585,246_1,Bagaimana tren penggunaan cuti karyawan selama...,Bagaimana distribusi penggunaan cuti karyawan ...,Pertanyaan ini memerlukan analisis segmentasi ...,"[attendances, attendance_statuses, employees, ...","SELECT \n DATE_FORMAT(a.date, '%Y-%m') AS m..."
586,246_2,Bagaimana tren penggunaan cuti karyawan selama...,Apa jenis cuti yang paling sering digunakan ol...,Pertanyaan ini memerlukan analisis komparatif ...,"[attendances, attendance_statuses, employees]",WITH employee_tenure AS (\n SELECT \n ...


In [2]:
import pandas as pd

ggg = pd.read_csv('augmented_sql_data_2/augmented_data_employee_complete.csv')
ggg

,No,Base Prompt,Prompt,SQL Reasoning,Required Tables,Expected SQL Query
0,53_1,Berapa jumlah karyawan aktif saat ini berdasar...,Bagaimana distribusi karyawan aktif saat ini b...,Pertanyaan ini memperluas analisis dengan mena...,"['employees', 'employee_details', 'employment_...","SELECT jl.name AS job_level, ed.gender, COUNT(..."
1,53_2,Berapa jumlah karyawan aktif saat ini berdasar...,Bagaimana tren perubahan jumlah karyawan aktif...,Pertanyaan ini menganalisis perubahan temporal...,"['employees', 'employee_details', 'employment_...",WITH monthly_counts AS (SELECT DATE_FORMAT(esh...
2,53_3,Berapa jumlah karyawan aktif saat ini berdasar...,Apakah ada perbedaan signifikan dalam rasio ge...,Pertanyaan ini membandingkan distribusi gender...,"['employees', 'employee_details', 'employment_...","SELECT et.name AS employment_type, ed.gender, ..."
3,54_1,Apa alasan utama karyawan mengundurkan diri pa...,Bagaimana tren pengunduran diri karyawan berub...,Pertanyaan ini memerlukan analisis temporal de...,"['termination_entries', 'termination_reasons']",SELECT \n QUARTER(termination_entries.effec...
4,54_2,Apa alasan utama karyawan mengundurkan diri pa...,Apakah ada perbedaan alasan pengunduran diri a...,Pertanyaan ini memerlukan analisis segmentasi ...,"['termination_entries', 'termination_reasons',...","SELECT \n employee_details.gender,\n ter..."
...,...,...,...,...,...,...
289,149_2,Berapa jumlah karyawan dengan kontrak yang aka...,Apa distribusi karyawan dengan kontrak yang ak...,Pertanyaan ini memerlukan analisis segmentasi ...,"['employment_statuses', 'employment_types', 'j...","SELECT \n et.name AS contract_type,\n jl..."
290,149_3,Berapa jumlah karyawan dengan kontrak yang aka...,Berapa persentase karyawan dengan kontrak yang...,Pertanyaan ini memerlukan analisis komparatif ...,"['employment_statuses', 'employment_status_typ...",WITH contract_counts AS (\n SELECT \n ...
291,150_1,Tampilkan hierarki manajemen lengkap untuk sel...,Bagaimana distribusi gender di setiap level ja...,Pertanyaan ini memerlukan analisis segmentasi ...,"['employees', 'employment_statuses', 'job_leve...",WITH RECURSIVE org_hierarchy AS (\n SELECT e....
292,150_2,Tampilkan hierarki manajemen lengkap untuk sel...,Bagaimana rata-rata masa kerja (tenure) karyaw...,Pertanyaan ini memerlukan analisis temporal un...,"['employees', 'employment_statuses', 'job_leve...",WITH RECURSIVE org_hierarchy AS (\n SELECT e....


## Upload to Google Sheets

In [69]:
df_syntetic_data = pd.read_csv('syntetics_data/syntetic_data_150_2.csv')
df_syntetic_data.head()

,no,category,complexity,question,required_tables,sql_concepts
0,1,TURNOVER ANALYSIS,Intermediate,Berapa persentase karyawan yang mengajukan pen...,"['termination_entries', 'termination_reasons']","['JOIN', 'COUNT', 'WHERE', 'GROUP BY']"
1,2,EMPLOYEE HIRING PATTERNS,Beginner,Berapa jumlah karyawan baru yang direkrut seti...,['employees'],"['COUNT', 'GROUP BY', 'DATE_FORMAT']"
2,3,MANAGEMENT HIERARCHY,Intermediate,Siapa manajer dengan jumlah bawahan langsung t...,['employees'],"['JOIN', 'COUNT', 'GROUP BY', 'ORDER BY']"
3,4,ATTENDANCE AND TIME TRACKING,Advanced,Bagaimana pola ketidakhadiran karyawan setelah...,"['employee_variables', 'employee_variable_meta...","['JOIN', 'WHERE', 'DATE', 'LIKE']"
4,5,EMPLOYMENT STATUS CHANGES,Intermediate,Berapa jumlah karyawan yang dipromosikan pada ...,"['employment_status_histories', 'employment_st...","['JOIN', 'COUNT', 'WHERE']"


In [74]:
from modules.google_sheets_writer import GoogleSheetsWriter
import logging


writer = GoogleSheetsWriter(
    google_util=google,  # Your GoogleUtil instance
    sheet_id=GOOGLE_SPREADSHEET_ID,
    worksheet_name="catapa_syntetics_question",
    batch_size=10,  # Customize batch size
    max_retries=5,  # Customize retry attempts
    batch_delay=2  # Customize delay between batches
)
# Write the DataFrame
result = writer.write_dataframe(df_syntetic_data)

# Log results
logging.info(f"Successfully wrote {result.successful_rows} rows")
if result.failed_rows > 0:
    logging.error(f"Failed to write {result.failed_rows} rows")
    for error in result.errors:
        logging.error(f"Row {error['row_number']}: {error['error']}")

  0%|          | 0/15 [00:00<?, ?it/s]2025-03-06 13:51:08,571 - INFO - Successfully wrote row 1/150
2025-03-06 13:51:10,832 - INFO - Successfully wrote row 2/150
2025-03-06 13:51:13,200 - INFO - Successfully wrote row 3/150
2025-03-06 13:51:15,741 - INFO - Successfully wrote row 4/150
2025-03-06 13:51:18,000 - INFO - Successfully wrote row 5/150
2025-03-06 13:51:20,113 - INFO - Successfully wrote row 6/150
2025-03-06 13:51:22,396 - INFO - Successfully wrote row 7/150
2025-03-06 13:51:24,574 - INFO - Successfully wrote row 8/150
2025-03-06 13:51:26,962 - INFO - Successfully wrote row 9/150
2025-03-06 13:51:29,229 - INFO - Successfully wrote row 10/150
  7%|▋         | 1/15 [00:25<05:54, 25.35s/it]2025-03-06 13:51:33,455 - INFO - Successfully wrote row 11/150
2025-03-06 13:51:35,713 - INFO - Successfully wrote row 12/150
2025-03-06 13:51:38,071 - INFO - Successfully wrote row 13/150
2025-03-06 13:51:40,249 - INFO - Successfully wrote row 14/150
2025-03-06 13:51:42,474 - INFO - Successful